In [ ]:
from numpy import array,float64,int64  
from matplotlib import pyplot
from pandas import read_csv,get_dummies
from pandas import DataFrame
from pandas import concat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
path = os.path.dirname(os.getcwd())

df_train = load_dataset(path + '/data/2015_train.csv')
df_test = load_dataset(path + '/data/2015_test.csv')

normalize_dataframes(df_train,df_test)

train_set = get_df_values(df_train) 
test_set = get_df_values(df_test)

X,y = split_sequences(sequences=train_set, n_steps = 10,)

model = compile_model(n_steps_in = 10,n_features_in = 51)
print(model.summary())

history = train_model(model=model,X=X,y=y,_validation_split = 0.3,_epoch=100,_batch_size=32,
_modelCheckpoint_path= path + '/MPS/TempNormalized',_modelCheckpoint_monitor='val_accuracy')



In [ ]:
def load_dataset(name):
    df = read_csv(name)
    df.drop(['Timestamp','Status'],axis=1, inplace=True)
    return df

In [ ]:
def normalize_dataframes(df_train,df_test):
    sc = MinMaxScaler(feature_range = (0, 1))

    df_train.loc[:, df_train.dtypes == float64] = sc.fit_transform(df_train.loc[:, df_train.dtypes == float64])
    df_test.loc[:, df_test.dtypes == float64] = sc.transform(df_test.loc[:, df_test.dtypes == float64])

In [ ]:
def get_df_values(df):
    return df.values

In [ ]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
def compile_model(n_steps_in,n_features_in):
    # define model
    model = Sequential()
    model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features_in)))
    model.add(LSTM(300, activation='relu',return_sequences=True))
    model.add(LSTM(200, activation='relu'))
    model.add(Dense(n_features_in,activation='relu'))
    model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
    return model



In [ ]:
def train_model(model,X,y,_validation_split,_epoch,_batch_size,_modelCheckpoint_path,_modelCheckpoint_monitor):
    return model.fit(X,y,validation_split=_validation_split, epochs = _epoch, batch_size = _batch_size, callbacks= [EarlyStopping(monitor='val_accuracy',patience=10,verbose=1), ModelCheckpoint(_modelCheckpoint_path, monitor=_modelCheckpoint_monitor,save_best_only=True)], verbose=1)

In [ ]:
def predict_from_excel(_excel_row,n_steps_in,_features_in):
    predvidanje = _excel_row
    x_input = test_set[predvidanje-13:predvidanje-3]
    
    x_input = x_input.reshape((1, n_steps_in, _features_in))
    yhat = model.predict(x_input, verbose=1)
    return yhat


In [ ]:
def plot_prediction(yhat,actual):
    pyplot.figure(figsize=(100,50))
    pyplot.plot(yhat, label='predict')
    pyplot.plot(actual, label='actual')

    pyplot.legend()
    pyplot.show()

In [ ]:
prediction = []
actual = []
for i in range(10):
    yhat = predict_from_excel(100000+i,10,51)
    prediction.append(yhat[0])
    actual.append(test_set[100000+i-2])

In [ ]:
#plot_prediction(yhat[0],actual[0])
actual[0]